<a href="https://colab.research.google.com/github/hulyacobans/weekly-challenges/blob/main/URL_Migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. GA Export**

The most difficult part in this script is to get the API key. If you can do that, the rest of the work you should do again SHIFT + ENTER! So here is a great guide how to do that: https://www.jcchouinard.com/google-api/ Thanks for this detailed guide Jean-Christophe :) 

GA code source: https://www.ilkkapeltola.fi/2018/02/know-exactly-how-much-you-pay-to.html

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Now, please put your client secret json file in your drive and copy URL path

In [ ]:
link = 'https://drive.google.com/open?id=your-url-path-for-client-secret-file'
fluff, id = link.split('=')
print (id)

In [ ]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('clientsecret.json') #what's your json file's name? If it is different than clientsecret.json, please change

In [ ]:
pip install --upgrade google-api-python-client

In [ ]:
import argparse
from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
import pandas as pd

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
CLIENT_SECRETS_PATH = 'clientsecret.json' 
VIEW_ID = 'xxxxxxxxxx' # change view ID

In [ ]:
query = {
    'reportRequests': [{
        'viewId' : 'YOUR VIEW ID',
        'pageSize' : 10000,
        
        'dateRanges': [{'startDate': '90daysAgo', 'endDate': 'today'}],
        'dimensions': [{'name': 'ga:landingPagePath'}],
        'metrics': [{'expression': 'ga:sessions'}],
        "orderBys": [
          {
          "fieldName": "ga:sessions",
          "sortOrder": "DESCENDING"
          }
        ],
        "pageSize": 10000,
        'dimensionFilterClauses': [
         {
          'filters': [
            {
            "dimensionName": "ga:medium",
            "expressions": ["organic"]
            }
          ]
        }
      ]
        }]
      }

In [ ]:
def initialize_analyticsreporting():
    parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
    flags = parser.parse_args([])
    flow = client.flow_from_clientsecrets(
      CLIENT_SECRETS_PATH, scope=SCOPES,
      message=tools.message_if_missing(CLIENT_SECRETS_PATH))
    storage = file.Storage('analyticsreporting.dat')
    credentials = storage.get()
    if credentials is None or credentials.invalid:
        credentials = tools.run_flow(flow, storage, flags)
    http = credentials.authorize(http=httplib2.Http())
    analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)
    return analytics

def get_report(analytics, query):
    nextPageToken = '0'
    responses = []
    while (nextPageToken != None):
        query['reportRequests'][0]['pageToken'] = nextPageToken
        result = analytics.reports().batchGet(
          body=query
        ).execute()
        responses.append(result)
        if 'nextPageToken' in result['reports'][0]: 
            nextPageToken = result['reports'][0]['nextPageToken']
        else:
            return responses
        
    return responses
    
def strip_ga_prefix(string):
    if string[:3] == 'ga:':
        return string[3:]
    else:
        return string


def get_dataframe(responses):
  rowlist = []
  for response in responses:
      for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

        for row in report.get('data', {}).get('rows', []):
          new_row = {}
          dimensions = row.get('dimensions', [])
          dateRangeValues = row.get('metrics', [])

          for header, dimension in zip(dimensionHeaders, dimensions):
            new_row[strip_ga_prefix(header)] = dimension

          for i, values in enumerate(dateRangeValues):
            for metricHeader, value in zip(metricHeaders, values.get('values')):
              new_row[strip_ga_prefix(metricHeader.get('name'))] = value

          rowlist.append(new_row)

  df3 = pd.DataFrame(rowlist)
  return df3

In [ ]:
analytics = initialize_analyticsreporting()
response = get_report(analytics, query3)
df3 = get_dataframe(response)

In [ ]:
df3.rename(columns = {'sessions': 'organic_sessions'}, inplace=True)
df3.rename(columns={'landingPagePath': 'URL'}, inplace=True)
df3['URL'] = 'https://' + df3['URL'].astype(str)
df3.head()

# **2. GSC Data**

GSC code source: https://sashadagayev.com/bulk-data-extraction-from-google-search-console-using-jupyter/

In [ ]:
import httplib2
from apiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow
CLIENT_ID = "YOUR-CLIENt-ID.apps.googleusercontent.com"
CLIENT_SECRET = "YOURCLIENTSECRET"

In [ ]:
OAUTH_SCOPE = 'https://www.googleapis.com/auth/webmasters.readonly'
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print('Go to the following link in your browser: ' + authorize_url)
code = input('Enter verification code: ').strip()
credentials = flow.step2_exchange(code)
http = httplib2.Http()
http = credentials.authorize(http)
webmasters_service = build('webmasters', 'v3', http=http)

In [ ]:
site = 'https://www.domain.com' #select domain
request = {
    'startDate': '2019-12-14', #all requests must include a start AND end date
    'endDate': '2020-02-14',
    'dimensions': ['page'], #select the dimensions of your response like ['date','query','page']
    #'dimensionsFilterGroups':[{'filters'}:[{'dimension':'device','expression':'mobile'}]] - this is sample filter
    'rowLimit':25000, #this is the maximum row limit for GSC export data
    'startRow':0
      }

In [ ]:
test_response = webmasters_service.searchanalytics().query(siteUrl=site,body=request).execute()
test_response

In [ ]:
def get_all_response(domain,request): #this is a function that agreggates responses for API calls over 25000 rows
    data = {'page':[], 'clicks':[],'impressions':[],'ctr':[], 'position':[]} #'date':[],'searchQuery':[],'page':[], 'clicks':[],'impressions':[],'ctr':[], 'position':[]
    startRow = 0
    request['startRow'] = startRow
    initresponse = webmasters_service.searchanalytics().query(siteUrl=site,body=request).execute()
    last_call_len = len(initresponse['rows'])
    mapped_dictionary = map_response_to_dict(initresponse, data)
    while last_call_len == 25000:
        print('this is running')
        startRow = startRow + 25000
        request['startRow'] = startRow
        response = webmasters_service.searchanalytics().query(siteUrl=site,body=request).execute()
        last_call_len = len(response['rows'])
        mapped_dictionary = map_response_to_dict(response, data)
    return mapped_dictionary

def map_response_to_dict(response, data): # this turns the JSON response into an easy to manipulate dictionary for dataframes
    for i in response['rows']:
        #date = i['keys'][0] #assumes that date is first
        #data['date'].append(date)
        #searchQuery = i['keys'][1] #assumes that query is second
        #data['searchQuery'].append(searchQuery)
        page = i['keys'][0] #assumes that page is third - if you add query -- [2]
        data['page'].append(page)
        clicks = i['clicks']
        data['clicks'].append(clicks)
        impressions = i['impressions']
        data['impressions'].append(impressions)
        ctr = i['ctr']
        data['ctr'].append(ctr)
        position = i['position']
        data['position'].append(position)
    return data

In [ ]:
big_dict = get_all_response(site,request)

In [ ]:
console_data = pd.DataFrame(big_dict)
console_data

# **3. MOZ TOP PAGES**

In [ ]:

from google.colab import files
uploaded = files.upload()

In [ ]:
moz = pd.read_csv('moz-top-pages-for-your-domain.csv', skiprows = 5, error_bad_lines=False )
moz

# **4. AHREFS LINK REPORT**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
ahrefs = pd.read_csv('yourdomain-best-pages-by-links-subdomains-17-Feb-2020_14-37-17.csv', encoding = 'utf-16', sep='\t')
ahrefs.head()

In [ ]:
#adding https:// 

moz['URL'] = 'https://' + moz['URL'].astype(str)
df['URL'] = 'https://' + df['URL'].astype(str)
moz.head()

In [ ]:
ahrefs2 = ahrefs[['URL', 'Referring Domains', 'Dofollow', 'Nofollow']]
moz2 = moz[['URL', 'Total Links', 'Linking Domains to Page']]

In [ ]:
merged = pd.merge(df3, console_data, on = 'URL', how = 'left')
merged2 = pd.merge(merged, moz2, on='URL', how = 'left')
merged3 = pd.merge(merged2, ahrefs2, on='URL', how = 'left')
merged3.rename(columns = {'Total Links': 'moz_total_links',
                          'Linking Domains to Page': 'moz_linking_domains',
                          'Referring Domains': 'ahrefs_referring_domains',
                          'Dofollow': 'dofollow',
                          'Nofollow': 'nofollow'}, inplace=True)
merged3.head()

In [ ]:
merged3.to_csv('filename.csv') 
files.download('filename.csv')